In [ ]:
!pip install scikit-image==0.19.3
!pip install pandas openpyxl
!pip install pyradiomics
!pip install mahotas
!pip install thefuzz[speedup]
!pip install scikit-learn opencv-python
!pip install numpy==1.23.5

In [ ]:
# Import standard libraries
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import shutil
import warnings

# Image processing libraries
from skimage import io, color, img_as_ubyte, filters
from skimage.color import rgb2gray
from skimage.filters import gabor
from skimage.feature import graycomatrix, graycoprops
from skimage.filters.rank import median
from skimage.morphology import disk, remove_small_objects, closing
from skimage.segmentation import chan_vese
from skimage.measure import regionprops, moments_central, label

# Statistical analysis libraries
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Machine learning and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Texture analysis libraries
import mahotas

!pip install SimpleITK
!pip install pyradiomics

# Radiomics libraries
import SimpleITK as sitk
from radiomics import featureextractor

# Ignore warnings
warnings.filterwarnings("ignore")

# Upload files
from google.colab import files


In [ ]:
def preprocess_image(img, depth=3.5):

    # Crop the image
    cropped_image = img

    # Resize the image to have height 500 pixels, maintaining aspect ratio
    h, w = cropped_image.shape[:2]
    new_h = 500
    new_w = int((new_h / h) * w)
    resized_img = cv2.resize(cropped_image, (new_w, new_h))

    # Compute mean luminance of the original image
    if len(resized_img.shape) == 3:
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    else:
        gray_img = resized_img

    mean_luminance_original = np.mean(gray_img)

    # Denoise using rank order filter (approximated using median filter)
    denoised_img = median(gray_img, disk(1))

    # Varying Clip Limit CLAHE
    tile_grid_size = (8, 8)
    clip_limit = 0.01

    # Adjust clip limit ratio based on mean luminance
    if mean_luminance_original < 80:
        clip_limit_ratio = 1.35  # Adjust as needed
    elif mean_luminance_original > 80:
        clip_limit_ratio = 1.05  # Adjust as needed
    else:
        clip_limit_ratio = 1.0

    initial_clip_limit_ratio = 1.0
    best_clip_limit = clip_limit

    # Apply CLAHE with varying clip limits
    while initial_clip_limit_ratio < clip_limit_ratio:
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
        clahe_img = clahe.apply(denoised_img)
        mean_luminance_clahe = np.mean(clahe_img)
        initial_clip_limit_ratio = mean_luminance_clahe / mean_luminance_original
        clip_limit += 0.001
        if clip_limit > 10:
            break  # Prevent infinite loop

    best_clip_limit = clip_limit

    # Apply CLAHE with the best clip limit
    clahe = cv2.createCLAHE(clipLimit=best_clip_limit, tileGridSize=tile_grid_size)
    clahe_img = clahe.apply(denoised_img)

    # Varying gamma
    min_ratio = 0.56  # Adjust as needed
    initial_gamma = 1.0
    ratio = 1.0

    while ratio > min_ratio:
        gamma_corrected = np.power(clahe_img / 255.0, initial_gamma) * 255.0
        gamma_corrected = np.clip(gamma_corrected, 0, 255).astype(np.uint8)
        mean_luminance_gamma = np.mean(gamma_corrected)
        ratio = mean_luminance_gamma / mean_luminance_original
        initial_gamma += 0.001
        if initial_gamma > 5:
            break  # Prevent infinite loop

    # Final gamma correction
    gamma_corrected_img = np.power(clahe_img / 255.0, initial_gamma) * 255.0
    gamma_corrected_img = np.clip(gamma_corrected_img, 0, 255).astype(np.uint8)

    # Create a three-channel grayscale image
    preprocessed_img = cv2.merge([gamma_corrected_img]*3)

    # K-means clustering
    double_image = preprocessed_img.astype(np.float64) / 255.0

    # Reshape the image into a 2D array for k-means
    reshaped_image = double_image.reshape(-1, 3)

    # Normalize the image data using RobustScaler
    scaler = RobustScaler()
    reshaped_image = scaler.fit_transform(reshaped_image)

    # Apply k-means clustering with k=2
    kmeans = KMeans(n_clusters=2, random_state=0).fit(reshaped_image)
    idx = kmeans.labels_

    # Reshape the clustered image back to its original size
    clustered_image = idx.reshape(preprocessed_img.shape[0], preprocessed_img.shape[1])

    # Segmentation using Chan-Vese method
    img_float = clustered_image.astype(np.float64)

    # Initial mask for active contour
    mask = np.zeros_like(img_float)
    mask[25:-25, 25:-25] = 1  # Define an initial mask

    # Run the active contour model (Snake) with 'Chan-Vese' method
    cv_result = chan_vese(img_float, max_iter=3000, init_level_set=mask, tol=1e-3)

    # Remove small areas
    min_area = 550  # Adjust this threshold as needed
    cv_cleaned = remove_small_objects(cv_result.astype(bool), min_size=min_area)

    # Morphological Closing
    selem = disk(5)  # Adjust the disk size based on the gap between regions
    mask_closed = closing(cv_cleaned, selem)

    # Remove small areas again
    mask_cleaned = remove_small_objects(mask_closed.astype(bool), min_size=min_area)

    # Detect lowest point
    y_coords, x_coords = np.nonzero(mask_cleaned)
    if len(y_coords) == 0:
        minX, minY = None, None
    else:
        maxY = np.max(y_coords)
        indices = np.where(y_coords == maxY)
        minX = x_coords[indices][0]
        minY = y_coords[indices][0]

    print(f"Minimum x value: {minX}")
    print(f"Minimum y value: {minY}")

    return preprocessed_img  # Return the preprocessed image


In [ ]:
def extract_morphological_features(image):
    if len(image.shape) == 3:  # Convert RGB image to grayscale
        image = color.rgb2gray(image)

    # Threshold the image using Otsu's method
    threshold = filters.threshold_otsu(image)
    binary_image = image > threshold

    # Label connected regions in the binary image
    labeled_image = label(binary_image)
    regions = regionprops(labeled_image)

    if not regions:
        return None  # Return None if no regions are found

    # Select the largest connected region
    largest_region = max(regions, key=lambda region: region.area)
    region_image = largest_region.image.astype(np.float64)

    # Compute central moments
    moments = moments_central(region_image)

    # Extract desired morphological features
    features = {
        'Area': largest_region.area,
        'Eccentricity': largest_region.eccentricity,
        'Perimeter': largest_region.perimeter,
        'Centroid_X': largest_region.centroid[1],  # X-coordinate
        'Centroid_Y': largest_region.centroid[0],  # Y-coordinate
        'Moments': moments  # Store the entire moments matrix
    }

    return features


In [ ]:
def format_value(v):# to handle scalars in title string
    # Check if v is a scalar.
    if np.isscalar(v):
        # Use pd.isna for scalar values.
        if pd.isna(v):
            return 'NaN'
        # Format floats with 4 decimals.
        elif isinstance(v, float):
            return f"{v:.4f}"
        else:
            return str(v)
    else:
        # If v is not scalar (e.g. an array), convert it to a string
        # using numpy's array2string with a chosen precision.
        return np.array2string(np.array(v), precision=4, separator=', ')


In [ ]:
# Upload image and Excel file
print("Please upload the Excel file with labels:")
legend = files.upload()  # Excel file with labels

In [ ]:
print("\nPlease upload the images:")
uploaded = files.upload()  # Images

In [ ]:
import re
import pandas as pd

# Organize images based on status from uploaded variable
images_by_status = {}

# Load the Excel legend
legend_filename = next(iter(legend.keys()))  # Get the uploaded Excel filename
legend_df = pd.read_excel(legend_filename, sheet_name='Sheet1')

# Ensure the legend has the expected columns
assert 'Code Number' in legend_df.columns, "The Excel file must have a 'Code Number' column."
assert 'Status' in legend_df.columns, "The Excel file must have a 'Status' column."

# Normalize the 'Code Number' column to strings for consistent matching
legend_df['Code Number'] = legend_df['Code Number'].astype(str).str.strip()

# Map codes to their corresponding statuses, using the last 4 digits for matching
codes_to_status = {str(int(code))[-4:]: int(status) for code, status in zip(legend_df['Code Number'], legend_df['Status'])}

# Debugging: Print all codes and their statuses
print("\nCodes and corresponding statuses from the legend (using last 4 digits):")
for code_suffix, status in codes_to_status.items():
    print(f"Code suffix: {code_suffix}, Status: {status}")

# Variables to hold images based on statuses
status_0 = []
status_1 = []
status_2 = []
status_3 = []

# Categorize images by status
for image_name, image_content in uploaded.items():
    matched = False
    # Extract numeric substrings from the image name
    numeric_substrings = re.findall(r'\d+', image_name)
    # Process each numeric substring
    for num_str in numeric_substrings:
        num_suffix = str(int(num_str))[-4:]  # Convert to int to strip leading zeros, then get last 4 digits
        if num_suffix in codes_to_status:
            status = codes_to_status[num_suffix]
            if status == 0:
                status_0.append((image_name, image_content))
            elif status == 1:
                status_1.append((image_name, image_content))
            elif status == 2:
                status_2.append((image_name, image_content))
            elif status == 3:
                status_3.append((image_name, image_content))
            matched = True
            break  # Stop after the first match
    if not matched:
        print(f"Image '{image_name}' did not match any code.")

# Debugging: Display the categorized images
print("\nImages categorized by status:")
print(f"Status 0: {len(status_0)} images")
print(f"Status 1: {len(status_1)} images")
print(f"Status 2: {len(status_2)} images")
print(f"Status 3: {len(status_3)} images")


In [ ]:
import re
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Organize images based on status from uploaded variable
images_by_status = {}

# Load the Excel legend
legend_filename = next(iter(legend.keys()))  # Get the uploaded Excel filename
legend_df = pd.read_excel(legend_filename, sheet_name='Sheet1')

# Ensure the legend has the expected columns
assert 'Code Number' in legend_df.columns, "The Excel file must have a 'Code Number' column."
assert 'Status' in legend_df.columns, "The Excel file must have a 'Status' column."

# Normalize the 'Code Number' column to strings for consistent matching
legend_df['Code Number'] = legend_df['Code Number'].astype(str).str.strip()

# Map codes to their corresponding statuses, using the last 4 digits for matching
codes_to_status = {str(int(code))[-4:]: int(status) for code, status in zip(legend_df['Code Number'], legend_df['Status'])}

# Debugging: Print all codes and their statuses
print("\nCodes and corresponding statuses from the legend (using last 4 digits):")
for code_suffix, status in codes_to_status.items():
    print(f"Code suffix: {code_suffix}, Status: {status}")

# Variables to hold images based on statuses
status_0 = []
status_1 = []
status_2 = []
status_3 = []

# Categorize images by status
for image_name, image_content in uploaded.items():
    matched = False
    # Extract numeric substrings from the image name
    numeric_substrings = re.findall(r'\d+', image_name)
    # Process each numeric substring
    for num_str in numeric_substrings:
        num_suffix = str(int(num_str))[-4:]  # Convert to int to strip leading zeros, then get last 4 digits
        if num_suffix in codes_to_status:
            status = codes_to_status[num_suffix]
            if status == 0:
                status_0.append((image_name, image_content))
            elif status == 1:
                status_1.append((image_name, image_content))
            elif status == 2:
                status_2.append((image_name, image_content))
            elif status == 3:
                status_3.append((image_name, image_content))
            matched = True
            break  # Stop after the first match
    if not matched:
        print(f"Image '{image_name}' did not match any code.")

# Debugging: Display the categorized images
print("\nImages categorized by status:")
print(f"Status 0: {len(status_0)} images")
print(f"Status 1: {len(status_1)} images")
print(f"Status 2: {len(status_2)} images")
print(f"Status 3: {len(status_3)} images")

# Display images using a slider
def display_image(status_list, idx):
    if idx < 0 or idx >= len(status_list):
        print("Invalid index")
        return
    image_name, image_content = status_list[idx]
    print(f"Image Name: {image_name}")
    image_array = np.frombuffer(image_content, dtype=np.uint8)
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    if img is not None:
        plt.figure(figsize=(6, 6))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Filename: {image_name}")
        plt.axis("off")
        plt.show()
    else:
        print(f"Unable to decode image: {image_name}")

# Interactive sliders for each status
def create_slider(status_list, status_label):
    if len(status_list) == 0:
        print(f"No images to display for Status {status_label}.")
        return
    print(f"\nInteractive slider for Status {status_label}:")
    interact(lambda idx: display_image(status_list, idx), idx=widgets.IntSlider(min=0, max=len(status_list)-1, step=1, description=f"Status {status_label}"))

# Create sliders for each status list
create_slider(status_0, 0)
create_slider(status_1, 1)
create_slider(status_2, 2)
create_slider(status_3, 3)


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import cv2
import SimpleITK as sitk
from skimage.feature import graycomatrix, graycoprops
from radiomics import featureextractor
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skew, kurtosis
from skimage.measure import moments, moments_central, moments_normalized, moments_hu

# Initialize variables for storing categorized image content
status_images = {0: [], 1: [], 2: [], 3: []}
status_filenames = {0: [], 1: [], 2: [], 3: []}

# Ensure 'legend' contains the Excel data
if 'legend' not in locals():
    raise ValueError("The 'legend' variable must be loaded with the Excel data.")

try:
    # Get the uploaded Excel filename from the 'legend' dictionary
    legend_filename = next(iter(legend.keys()))

    # Read the Excel file into a DataFrame
    legend_df = pd.read_excel(legend_filename, sheet_name='Sheet1')

    # Ensure 'Code Number' is a string and strip whitespace
    legend_df['Code Number'] = legend_df['Code Number'].astype(str).str.strip()

    # Extract the first 6 digits from 'Code Number' to create a mapping
    legend_df['Code_Number_6digits'] = legend_df['Code Number'].apply(
        lambda x: re.search(r'\b(\d{6})\b', x).group(1) if re.search(r'\b(\d{6})\b', x) else None)

    # Create a mapping from code numbers to their features
    code_number_to_features = legend_df.set_index('Code_Number_6digits').to_dict(orient='index')

except Exception as e:
    raise ValueError(f"Error processing 'legend' variable: {e}")

# Function to extract the first 6-digit code number from filename
def extract_code_number(filename):
    filename_no_ext = os.path.splitext(filename)[0]
    match = re.search(r'\b(\d{6})\b', filename_no_ext)
    return match.group(1) if match else None

# Categorize images by status
if 'uploaded' in locals():
    for image_name, image_content in uploaded.items():
        extracted_code_number = extract_code_number(image_name)

        if extracted_code_number and extracted_code_number in code_number_to_features:
            status = code_number_to_features[extracted_code_number].get('Status', None)

            if status in status_images:
                status_images[status].append(image_content)
                status_filenames[status].append(image_name)
        else:
            print(f"Image '{image_name}' did not match any code.")

# Combine all status variables into a dictionary for dynamic processing
results = []

# Process images by their status
for status, image_list in status_images.items():
    for idx, image_content in enumerate(image_list):
        original_filename = status_filenames[status][idx]

        # Read the image from the content
        image_array = np.frombuffer(image_content, dtype=np.uint8)
        img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

        if img is not None:
            # Preprocess the image
            img_prep = preprocess_image(img)

            # Convert to grayscale
            img_gray = cv2.cvtColor(img_prep, cv2.COLOR_BGR2GRAY) if img_prep.ndim == 3 else img_prep.copy()

            # Convert the image to uint8 format
            img_uint8 = img_gray.astype(np.uint8)
            img_uint8 = cv2.normalize(img_uint8, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

            # Convert the NumPy image to SimpleITK image
            sitk_image = sitk.GetImageFromArray(img_uint8)

            # Create a mask with ones (foreground)
            mask = np.ones(img_uint8.shape, dtype=np.uint8)
            mask[0, :] = mask[-1, :] = mask[:, 0] = mask[:, -1] = 0
            sitk_mask = sitk.GetImageFromArray(mask)

            # Extract additional features from column "C" onwards for the matching code number
            matching_code_number = extract_code_number(original_filename)  # Use the provided function to extract code number
            additional_features = {}

            # Check if the code number exists in the legend
            if matching_code_number in legend_df['Code Number'].astype(str).values:
                # Extract the row matching the code number
                matching_row = legend_df.loc[legend_df['Code Number'].astype(str) == matching_code_number]

                # Extract columns from "C" (index 2 in 0-based indexing) onwards
                for col in legend_df.columns[2:]:
                    additional_features[col] = matching_row[col].values[0]  # Extract the value

            # Extract GLCM features
            try:
                distances = [1]
                angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
                glcm = graycomatrix(img_uint8, distances=distances, angles=angles, symmetric=True, normed=True)

                # Dissimilarity for each angle
                diss_sim_0   = graycoprops(glcm, 'dissimilarity')[0, 0]
                diss_sim_45  = graycoprops(glcm, 'dissimilarity')[0, 1]
                diss_sim_90  = graycoprops(glcm, 'dissimilarity')[0, 2]
                diss_sim_135 = graycoprops(glcm, 'dissimilarity')[0, 3]

                # Correlation for each angle
                corr_0   = graycoprops(glcm, 'correlation')[0, 0]
                corr_45  = graycoprops(glcm, 'correlation')[0, 1]
                corr_90  = graycoprops(glcm, 'correlation')[0, 2]
                corr_135 = graycoprops(glcm, 'correlation')[0, 3]

                # Homogeneity for each angle
                homogen_0   = graycoprops(glcm, 'homogeneity')[0, 0]
                homogen_45  = graycoprops(glcm, 'homogeneity')[0, 1]
                homogen_90  = graycoprops(glcm, 'homogeneity')[0, 2]
                homogen_135 = graycoprops(glcm, 'homogeneity')[0, 3]

                # Energy for each angle
                energy_0   = graycoprops(glcm, 'energy')[0, 0]
                energy_45  = graycoprops(glcm, 'energy')[0, 1]
                energy_90  = graycoprops(glcm, 'energy')[0, 2]
                energy_135 = graycoprops(glcm, 'energy')[0, 3]

                # Contrast for each angle
                contrast_0   = graycoprops(glcm, 'contrast')[0, 0]
                contrast_45  = graycoprops(glcm, 'contrast')[0, 1]
                contrast_90  = graycoprops(glcm, 'contrast')[0, 2]
                contrast_135 = graycoprops(glcm, 'contrast')[0, 3]

            except Exception as e:
                print(f"Error computing GLCM features for '{original_filename}': {e}")
                diss_sim_0 = diss_sim_45 = diss_sim_90 = diss_sim_135 = np.nan
                corr_0 = corr_45 = corr_90 = corr_135 = np.nan
                homogen_0 = homogen_45 = homogen_90 = homogen_135 = np.nan
                energy_0 = energy_45 = energy_90 = energy_135 = np.nan
                contrast_0 = contrast_45 = contrast_90 = contrast_135 = np.nan




            # Extract GLRLM features using PyRadiomics
            try:
                extractor = featureextractor.RadiomicsFeatureExtractor()
                extractor.disableAllFeatures()
                extractor.enableFeatureClassByName('glrlm')
                glrlm_features = extractor.execute(sitk_image, sitk_mask)

                run_length_non_uniformity = glrlm_features.get('original_glrlm_RunLengthNonUniformity', np.nan)
                short_run_emphasis = glrlm_features.get('original_glrlm_ShortRunEmphasis', np.nan)
                long_run_emphasis = glrlm_features.get('original_glrlm_LongRunEmphasis', np.nan)
                gray_level_non_uniformity = glrlm_features.get('original_glrlm_GrayLevelNonUniformity', np.nan)
                run_percentage = glrlm_features.get('original_glrlm_RunPercentage', np.nan)
                run_entropy = glrlm_features.get('original_glrlm_RunEntropy', np.nan)
            except Exception as e:
                print(f"Error computing GLRLM features for '{original_filename}': {e}")
                run_length_non_uniformity = short_run_emphasis = long_run_emphasis = np.nan
                gray_level_non_uniformity = run_percentage = run_entropy = np.nan

            try:
                    # Extract morphological features
                    morphological_features = extract_morphological_features(img_gray)

                    if morphological_features:
                        # Extract morphological features
                        area = morphological_features['Area']
                        eccentricity = morphological_features['Eccentricity']
                        perimeter = morphological_features['Perimeter']
                        centroid_x = morphological_features['Centroid_X']
                        centroid_y = morphological_features['Centroid_Y']


                    else:
                        # Handle cases where no morphological features are extracted
                        area = eccentricity = perimeter = centroid_x = centroid_y = np.nan


            except Exception as e:
                    # Handle exceptions gracefully and set default NaN values
                    print(f"Error computing morphological features for '{original_filename}': {e}")
                    area = eccentricity = perimeter = centroid_x = centroid_y = np.nan


            mu = moments_central(img_gray)
            mu00 = mu[0, 0]
            mu01 = mu[0, 1]
            mu02 = mu[0, 2]
            mu10 = mu[1, 0]
            mu11 = mu[1, 1]
            mu12 = mu[1, 2]
            mu20 = mu[2, 0]
            mu21 = mu[2, 1]
            mu22 = mu[2, 2]

            nu = moments_normalized(mu)
            nu00 = nu[0, 0]
            nu01 = nu[0, 1]
            nu02 = nu[0, 2]
            nu10 = nu[1, 0]
            nu11 = nu[1, 1]
            nu12 = nu[1, 2]
            nu20 = nu[2, 0]
            nu21 = nu[2, 1]
            nu22 = nu[2, 2]

            hu = moments_hu(nu)
            hu0 = hu[0]
            hu1 = hu[1]
            hu2 = hu[2]
            hu3 = hu[3]
            hu4 = hu[4]
            hu5 = hu[5]
            hu6 = hu[6]

            flat_image = img_gray.flatten()
            mean_img = np.mean(flat_image)
            std_img = np.std(flat_image)
            var_img = np.var(flat_image)
            skew_img = skew(flat_image)
            kurt_img = kurtosis(flat_image)  # Note: returns excess kurtosis


            # Append the results to the list

                        # Append the results to the list
            result_entry = {
                'Filename': original_filename,
                'Status': status,

                'Mean': mean_img,
                'Std': std_img,
                'Variance': var_img,
                'Skewness': skew_img,
                'Kurtosis': kurt_img,

                'Dissimilarity_0': diss_sim_0,
                'Dissimilarity_45': diss_sim_45,
                'Dissimilarity_90': diss_sim_90,
                'Dissimilarity_135': diss_sim_135,

                'Correlation_0': corr_0,
                'Correlation_45': corr_45,
                'Correlation_90': corr_90,
                'Correlation_135': corr_135,

                'Homogeneity_0': homogen_0,
                'Homogeneity_45': homogen_45,
                'Homogeneity_90': homogen_90,
                'Homogeneity_135': homogen_135,

                'Energy_0': energy_0,
                'Energy_45': energy_45,
                'Energy_90': energy_90,
                'Energy_135': energy_135,

                'Contrast_0': contrast_0,
                'Contrast_45': contrast_45,
                'Contrast_90': contrast_90,
                'Contrast_135': contrast_135,

                'RunLengthNonUniformity': run_length_non_uniformity,
                'ShortRunEmphasis': short_run_emphasis,
                'LongRunEmphasis': long_run_emphasis,
                'GrayLevelNonUniformity': gray_level_non_uniformity,
                'RunPercentage': run_percentage,
                'RunEntropy': run_entropy,

                'Area': area,
                'Eccentricity': eccentricity,
                'Perimeter': perimeter,
                'Centroid_X': centroid_x,
                'Centroid_Y': centroid_y,
                'Mu00': mu00, 'Mu01': mu01, 'Mu02': mu02,
                'Mu10': mu10, 'Mu11': mu11, 'Mu12': mu12,
                'Mu20': mu20, 'Mu21': mu21, 'Mu22': mu22,
                # Individual normalized moments:
                'Nu00': nu00, 'Nu01': nu01, 'Nu02': nu02,
                'Nu10': nu10, 'Nu11': nu11, 'Nu12': nu12,
                'Nu20': nu20, 'Nu21': nu21, 'Nu22': nu22,
                # Hu invariant moments:
                'Hu0': hu0, 'Hu1': hu1, 'Hu2': hu2,
                'Hu3': hu3, 'Hu4': hu4, 'Hu5': hu5, 'Hu6': hu6
            }

             # Add the additional features to the result entry
            result_entry.update(additional_features)


            # Visualization
            plt.figure(figsize=(12, 8))
            plt.imshow(img_gray, cmap='gray')
            plt.axis('off')


            # Build the title string with image features
            'title_str = f"Image: {original_filename}\nStatus: {status}"'
            title_str = ''

            for k, v in result_entry.items():
                v_str = format_value(v)
                title_str += f"\n{k}: {v_str}"
                title_str += f"\n{k}: {v_str}"

            plt.title(title_str)
            plt.show()


            # Append the result entry
            results.append(result_entry)
        else:
            print(f"Failed to load image: {original_filename}")


# Create a DataFrame and save the results
df = pd.DataFrame(results)
df.to_csv('extracted_features_with_prep.csv', index=False)
print("Features saved to 'extracted_features_with_prep.csv'")


In [ ]:
files.download('extracted_features_with_prep.csv')
